# Cadence Effects

This notebook simulates normal Type Ia Supernova (SN Ia) light-curves using realistic cadences and atmospheric variabilities expected from LSST.


In [ ]:
import sys
sys.path.insert(0, '../')

from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import sncosmo
from astropy.table import Table
from astropy.time import Time
from astropy.io import fits
from matplotlib import pyplot as plt
from pwv_kpno.package_settings import ConfigBuilder, settings
from pwv_kpno import pwv_atm
from pytz import utc

from sn_analysis import filters, plasticc, plotting, sn_magnitudes 
from sn_analysis.modeling import PWVSource

filters.register_lsst_filters(force=True)


In [ ]:
plt.rcParams['figure.dpi'] = 100


## Atmospheric Variability

To create a physically reasonable representation of the atmospheric variability at LSST, we use PWV measurements taken at the nearby Cerro Telolo International Observatory (CTIO).


In [ ]:
# This only needs to be run once in a given environment
try:
    settings.set_site('cerro_tololo')
    
except ValueError:
    ctio_config = ConfigBuilder(
        site_name='cerro_tololo',
        primary_rec='CTIO',
        sup_rec=[]
    )

    ctio_config.save_to_ecsv('./cerro_tololo.ecsv', overwrite=True)
    settings.import_site_config('./cerro_tololo.ecsv', overwrite=True)
    
    settings.set_site('cerro_tololo')
    pwv_atm.update_models(range(2012, 2019))


In [ ]:
ctio_pwv = pwv_atm.measured_pwv().to_pandas('date')

ctio_pwv.reset_index().plot.scatter('date', 'CTIO', s=1, figsize=(10, 4), alpha=.2)
plt.ylabel('CTIO PWV (mm)')
plt.xlabel('Date')
plt.title('All available PWV measurements for CTIO')
plt.ylim(0, 20)


We don't have enough data to fully represent a 10 year long survey. Fortunately we are mostly interested in timescales of seasonal variability and shorter so we can consider data from a single year with good measurement coverage. 

In [ ]:
good_pwv_data = ctio_pwv[ctio_pwv.CTIO <= 25]
plotting.plot_year_pwv_vs_time(good_pwv_data[good_pwv_data.index.year == 2016].CTIO)
plt.title('CTIO PWV over 2016');


Out of simple curiosity, we also visualize the median PWV across all available years.


In [ ]:
# Todo: This really should have been a modulo opperation

folded_pwv = good_pwv_data.copy()
folded_pwv.index = folded_pwv.index.map(lambda t: t.replace(year=2000))  # Use a dummy year
folded_pwv = folded_pwv.groupby(folded_pwv.index).CTIO.median()


In [ ]:
plotting.plot_year_pwv_vs_time(folded_pwv)
plt.title('Phase folded PWV');


## The PLaSTICC Data

Instead of evaluating different cadences from scratch, we use light-curves from the PLaSTICC simulations. First we check what cadence simulations are available on the notebook's host server.


In [ ]:
plasticc.get_available_cadences()


Simulated light-curves are written in the SNANA file format and are distributed across multiple files. We load a light-curve from one of these files and demosntrate the data model below. Each cadence includes simulations run with multiple supernova models. In this notebook we only need simulations for normal SNe (Model 11). 


In [ ]:
demo_cadence = 'alt_sched_rolling'
demo_cadence_header_files = plasticc.get_model_headers('alt_sched_rolling', 11)

demo_header_path = demo_cadence_header_files[0]
with fits.open(demo_header_path) as _temp:
    light_curves_per_file = len(_temp[1].data) 

print('Files per cadence:', len(demo_cadence_header_files))
print('Max light-curves per file:', light_curves_per_file)
    

In [ ]:
plasticc_lc = next(plasticc.iter_lc_for_header(demo_header_path, verbose=False))


In [ ]:
plasticc_lc.meta


In [ ]:
plasticc_lc


Here we reformat the data to be compatible with `sncosmo` so we can easily visualize the light-curve.


In [ ]:
formatted_lc = plasticc.format_plasticc_sncosmo(plasticc_lc)


In [ ]:
sncosmo.plot_lc(formatted_lc);


## Simulating Light-Curves

Since we need to add in our own atmospheric variability, the pre-tabulated flux values above are of limited use. Instead, we use the PLaSTICC light-curves to establish the cadence and model parameters for each simulated SN. This information is then used to simulate our own light-curves with `sncosmo`.


**Note:** See Issue 8 (https://github.com/LSSTDESC/SN-PWV/issues/8) for caveats about the following cell.

In [ ]:
duplicated_lc = plasticc.duplicate_plasticc_sncosmo(
    plasticc_lc, gain=20, skynr=100)


In [ ]:
duplicated_lc.meta


In [ ]:
sncosmo.plot_lc(duplicated_lc);


In [ ]:
duplicated_lc


## Adding PWV Variability

The `sncosmo` package doesn't have a clearly defined approach to adding time variabile propagation effects, so we impliment a slightly hacky solution. To make this work we impliment 
1. A function that returned the desired PWV as a function of time.
2. A child class of `sncosmo.Source` that incorporates PWV transmission effects by overloading the flux calculation of an existing sncosmo source.

We define the first of these below. The interpolator maps a given datetime onto the number of seconds that have elapsed since the start of the year. This number is used to interpolate the PWV from a single year's worth of PWV measurements. 

In [ ]:
def datetime_to_sec_in_year(dates):
    """Calculate number of seconds elapsed modulo 1 year
    
    Args:
        dates (pd.Datetime): Pandas datetime array
        
    Returns:
        A numpy array of integers
    """
    
    dates = pd.to_datetime(dates)
    
    hour_in_day = 24
    min_in_hour = sec_in_min = 60
    return (
        dates.dayofyear * hour_in_day * min_in_hour * sec_in_min
        + dates.hour * min_in_hour * sec_in_min
        + dates.minute * sec_in_min
    )


def build_interpolater_from_suomi_data(df, year, df_col='CTIO'):
    """Build interpolator for the PWV at a given point of the year
    
    Args:
        df (DataFrame): Dataframe with PWV values and a Datetime index
        year   (float): Year of data to use from ``df``
        df_col   (str): Name of column in `df` with PWV values
    """
    
    sample_data = df[df.index.year == year]
    
    def interp(mjd):
        x = Time(mjd, format='mjd').to_datetime()
        return np.interp(
            x=datetime_to_sec_in_year(x),
            xp=datetime_to_sec_in_year(sample_data.index),
            fp=sample_data[df_col]
        )
    
    return interp


In [ ]:
pwv_interpolator = build_interpolater_from_suomi_data(good_pwv_data, 2016)


We make a quick validation plot to validate the interpolation function.


In [ ]:
def plot_interpolation_validation(pwv_data, year, df_col='CTIO'):
    """Overplot the interpolated and measured PWV
    
    Args:
        df (DataFrame): Dataframe with PWV values and a Datetime index
        year   (float): Year of data to use from ``df``
        df_col   (str): Name of column in `df` with PWV values
    """
    
    pwv_interpolator = build_interpolater_from_suomi_data(pwv_data, year)
    
    test_df = pwv_data[pwv_data.index.year == year][:500].copy()
    interpolated_pwv = pwv_interpolator(Time(test_df.index).mjd)

    plt.figure(figsize=(9, 3))
    plt.scatter(test_df.index, test_df.CTIO, s=20, label='Measured')
    plt.plot(test_df.index, interpolated_pwv, label='Interpolated', color='C1')
    plt.ylabel('PWV (mm)')
    plt.xlabel('Date (UTC)')
    plt.legend()


In [ ]:
plot_interpolation_validation(good_pwv_data, 2016)


Next we build the SN model using a customized source. The `PWVSource` class wraps a given SNCosmo source and overloads the underlying flux calculation. Since source objects are not aware of any time metric (other than phase) we need to connect the underlying source object to the parent `Model` obect as follows:


> `model_with_pwv = sncosmo.Model(PWVSource('salt2-extended', pwv_interpolator))`
> `model_with_pwv.source.parent_model = model_with_pwv`

Note that because of this approach, the `t0` parameter for each light-curve is now in units of MJD (i.e., the same units as the interpolation function).


In [ ]:
def plot_variable_pwv_model(source, phase=0):
    """Overplot a sncosmo model with and without temporally variable PWV
    
    Args:
        source (str, Source): sncosmo source to plot
        phase        (float): Phase of the supernova to plot
    """
    
    wave = np.arange(6000, 12000)

    model_without_pwv = sncosmo.Model(source)
    flux_without_pwv = model_without_pwv.flux(phase, wave)

    model_with_pwv = sncosmo.Model(PWVSource(source, pwv_interpolator))
    model_with_pwv.source.parent_model = model_with_pwv
    flux_with_pwv = model_with_pwv.flux(phase, wave)

    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(wave, flux_without_pwv, label='Base source', color='C1')
    ax.plot(wave, flux_with_pwv, label='Source with PWV', color='C0')
    ax.set_title('Simulated Flux')
    ax.set_ylabel('Flux')
    ax.legend()
    ax.set_xlabel('Wavelength (A)')
    ax.set_xlim(min(wave), max(wave))

    plt.tight_layout()


In [ ]:
plot_variable_pwv_model('salt2-extended')


## Fitting Light-Curves

In [ ]:
def iter_custom_lcs(cadence, scatter=True, quality_callback=None, drop_nondetection=False, verbose=True):
    """Simulate light-curves for a given cadence
    
    Args:
        cadence               (str): Cadence to use when simulating light-curves
        scatter              (bool): Add random noise to the flux values
        quality_callback (callable): Skip light-curves if this function returns False
        drop_nondetection    (bool): Drop data with PHOTFLAG == 0
        verbose              (bool): Display a progress bar
    """
    
    model_with_pwv = sncosmo.Model(PWVSource(source, pwv_interpolator))
    model_with_pwv.source.parent_model = model_with_pwv
    
    cadence_header = plasticc.get_model_headers(cadence, model=11)[0]
    for light_curve in plasticc.iter_lc_for_header(cadence_header, verbose=verbose):
        
        lc = plasticc.duplicate_plasticc_sncosmo(light_curve, model_with_pwv, scatter=scatter)
        if quality_callback and not quality_callback(lc):
            continue
            
        yield lc 
        

In [ ]:
def passes_quality_cuts(light_curve):
    """Return whether light-curve has 2+ two bands each with 1+ data point with SNR > 5
    
    Args:
        light_curve (Table): Astropy table with sncosmo formatted light-curve data
        
    Returns:
        A boolean
    """
    
    light_curve = light_curve.group_by('band')
    
    passed_cuts = []
    for band_lc in light_curve.groups:
        passed_cuts.append((band_lc['flux'] /  band_lc['fluxerr'] > 5).any())
        
    return sum(passed_cuts) >= 2
        

**Todo:** This notebook is still under construction. Any cells below this line are not expected to work.

In [ ]:
bands = ['lsst_hardware_' + b for b in 'ugrizy']

# Iterator over simulated light-curves
light_curves = iter_custom_lcs('alt_sched_rolling', quality_callback=passes_quality_cuts)

# Fit light curves
vparams = ['x0', 'x1', 'c']
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
        'salt2', light_curves, vparams, bands=bands)
